In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import pandas as pd
import gc
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from IPython.display import clear_output
from transformers.utils import logging
logging.set_verbosity_error()
import warnings
warnings.filterwarnings(action="ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Chọn GPU nếu có

In [ ]:
pretrained_weights = 'NlpHUST/gpt2-vietnamese'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights).to(device) 

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [ ]:
path = ''
df = pd.read_csv(r'shopee_data/new_shopee_1.csv')
df.columns

In [ ]:
sample_df = df.rename(columns={'father_name': 'category', 'comment': 'reviewText'})
sample_df["sentenceLength"] = sample_df["reviewText"].apply(lambda text: len(tokenizer.encode(text)))
sample_df = sample_df[['category', 'reviewText', 'sentenceLength']]
sample_df.head()

In [ ]:
gc.collect()

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [ ]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples of Toys_and_Games_5

In [ ]:
model_finetuned_path = r"gpt2-finetuned-shopee"

learn.load(model_finetuned_path).cuda()
learn.model.to(device)

In [ ]:
N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 36000
BINS = list(range(5,300))

In [ ]:
generated_df_rows = []

for i in tqdm(range(len(BINS) - 1)):
    clear_output(wait=True)  
    sample_row_df = sample_df[
        (sample_df["sentenceLength"] >= BINS[i]) & (sample_df["sentenceLength"] < BINS[i + 1])
    ]
    bin_proportion = sample_row_df.shape[0] / float(LEN_SAMPLE_DF)
    n_reviews_to_generate = int(bin_proportion * N_REVIEWS) + 1
    if len(sample_row_df) < 2:
        continue    
    try:
        or_df, cg_df = train_test_split(sample_row_df, test_size=0.5, random_state=2021)

        sample_reviews_df = cg_df.sample(n=min(n_reviews_to_generate, len(cg_df)), random_state=2021)
        or_sample_reviews_df = or_df.sample(n=min(n_reviews_to_generate, len(or_df)), random_state=2021)
        
        max_len = BINS[i]
    
        for index, row in sample_reviews_df.iterrows():
            text = row["reviewText"]
            prompt = " ".join(text.strip().split(" ")[:N_INITIAL_WORDS])
            prompt_ids = tokenizer.encode(prompt)
            # inp = tensor(prompt_ids)[None].cuda()
            inp = torch.tensor(prompt_ids, device=device)[None]  # Chuyển tensor lên GPU

            random_seed(seed_value=2021, use_cuda=True)
            
            preds = learn.model.generate(inp, max_length=max_len + 1, do_sample=True, top_k=0, top_p=0.92, temperature=0.7)
            decoded_preds = tokenizer.decode(preds[0], skip_special_tokens=True)
            
            generated_df_rows.append([row["category"], decoded_preds, "CG"])
    
        for index, row in or_sample_reviews_df.iterrows():
            generated_df_rows.append([row["category"], row["reviewText"], "OR"])
            
    except Exception as e:
        print(e)
        continue

In [ ]:
generated_df = pd.DataFrame(generated_df_rows, columns=["category", "text", "label"])

In [ ]:
generated_df.shape

In [ ]:
generated_df.sample(20)

In [ ]:
generated_df.head()

In [ ]:
generated_df.to_csv("Generated_Sentences_1.csv", index=None)